In [1]:
import pandas as pd
import numpy as np
import re
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data_raw_rumah123.csv')
df.head()

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,Dilengkapi Perabotan,Kondisi Properti,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas
0,Rp. 25.000.000.000,"Senen, Jakarta Pusat",Luas Bangunan : 1553 m²,Luas Tanah : 774 m²,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,Furnished,Bagus Sekali,3.0,2.0,2.0,SHM - Sertifikat Hak Milik,1300,6 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN
1,Rp. 1.750.000.000,"Cempaka Putih, Jakarta Pusat",Luas Bangunan : 97 m²,Luas Tanah : 49 m²,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,Unfurnished,Bagus Sekali,3.0,NaN,NaN,SHM - Sertifikat Hak Milik,2200,27 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN
2,Rp. 1.500.000.000,"Sumur Batu, Jakarta Pusat",Luas Bangunan : 110 m²,Luas Tanah : 70 m²,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,Semi Furnished,Bagus,2.0,1.0,1.0,SHM - Sertifikat Hak Milik,2200,10 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN
3,Rp. 1.750.000.000,"Senen, Jakarta Pusat",Luas Bangunan : 150 m²,Luas Tanah : 84 m²,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,NaN,NaN,2.0,NaN,NaN,"Lainnya (PPJB,Girik,Adat,dll)",NaN,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN
4,Rp. 1.375.000.000,"Cempaka Putih, Jakarta Pusat",Luas Bangunan : 65 m²,Luas Tanah : 35 m²,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,Unfurnished,Baru,2.0,NaN,NaN,SHM - Sertifikat Hak Milik,2200,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9868 entries, 0 to 9867
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9868 non-null   object 
 1   lokasi                9868 non-null   object 
 2   lb                    9868 non-null   object 
 3   lt                    9868 non-null   object 
 4   kt                    9578 non-null   float64
 5   km                    9331 non-null   float64
 6   car                   3997 non-null   float64
 7   Deskripsi             9868 non-null   object 
 8   ID Iklan              9681 non-null   object 
 9   Tipe Properti         9681 non-null   object 
 10  Dilengkapi Perabotan  5752 non-null   object 
 11  Kondisi Properti      6579 non-null   object 
 12  Jumlah Lantai         7105 non-null   float64
 13  Kamar Pembantu        4154 non-null   float64
 14  Kamar Mandi Pembantu  3764 non-null   float64
 15  Sertifikat           

In [4]:
### Drop rows where most of its columns empty
df.dropna(subset=['ID Iklan'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9681 entries, 0 to 9867
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9681 non-null   object 
 1   lokasi                9681 non-null   object 
 2   lb                    9681 non-null   object 
 3   lt                    9681 non-null   object 
 4   kt                    9399 non-null   float64
 5   km                    9155 non-null   float64
 6   car                   3997 non-null   float64
 7   Deskripsi             9681 non-null   object 
 8   ID Iklan              9681 non-null   object 
 9   Tipe Properti         9681 non-null   object 
 10  Dilengkapi Perabotan  5752 non-null   object 
 11  Kondisi Properti      6579 non-null   object 
 12  Jumlah Lantai         7105 non-null   float64
 13  Kamar Pembantu        4154 non-null   float64
 14  Kamar Mandi Pembantu  3764 non-null   float64
 15  Sertifikat           

### Munging Kolom Harga

In [5]:
df['harga'].sample(5)

6429     Rp. 1.200.000.000
8510     Rp. 5.500.000.000
703      Rp. 2.000.000.000
6092     Rp. 8.000.000.000
1522    Rp. 36.000.000.000
Name: harga, dtype: object

In [6]:
def AmbilAngka(text):
    angka = ''.join(re.findall(r'[^\D]+',text))
    return angka
df['harga'] = df['harga'].apply(lambda x : AmbilAngka(x))
df['harga'].sample(10)

7722       70000000
5408     3000000000
9132    16000000000
9776      500000000
759      1800000000
3166      500000000
5625     4000000000
6840     1700000000
9715    22000000000
1529     3800000000
Name: harga, dtype: object

In [7]:
df.head(2)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,Dilengkapi Perabotan,Kondisi Properti,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas
0,25000000000,"Senen, Jakarta Pusat",Luas Bangunan : 1553 m²,Luas Tanah : 774 m²,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,Furnished,Bagus Sekali,3.0,2.0,2.0,SHM - Sertifikat Hak Milik,1300,6 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN
1,1750000000,"Cempaka Putih, Jakarta Pusat",Luas Bangunan : 97 m²,Luas Tanah : 49 m²,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,Unfurnished,Bagus Sekali,3.0,NaN,NaN,SHM - Sertifikat Hak Milik,2200,27 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN


In [8]:
# df['harga'] = pd.to_numeric(df['harga'], downcast="float")
df['harga'] = df['harga'].astype(float)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9681 entries, 0 to 9867
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9681 non-null   float64
 1   lokasi                9681 non-null   object 
 2   lb                    9681 non-null   object 
 3   lt                    9681 non-null   object 
 4   kt                    9399 non-null   float64
 5   km                    9155 non-null   float64
 6   car                   3997 non-null   float64
 7   Deskripsi             9681 non-null   object 
 8   ID Iklan              9681 non-null   object 
 9   Tipe Properti         9681 non-null   object 
 10  Dilengkapi Perabotan  5752 non-null   object 
 11  Kondisi Properti      6579 non-null   object 
 12  Jumlah Lantai         7105 non-null   float64
 13  Kamar Pembantu        4154 non-null   float64
 14  Kamar Mandi Pembantu  3764 non-null   float64
 15  Sertifikat           

### Munging Kolom Lokasi

In [10]:
df['lokasi_copy'] = df['lokasi']

In [11]:
df.sample(1)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Kondisi Properti,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy
3736,4.500000e+09,"Cipinang, Jakarta Timur",Luas Bangunan : 260 m²,Luas Tanah : 175 m²,4.0,3.0,2.0,Rumah Siap Huni Cipinang Jakarta Timur:DiJual\...,hos7652279,Rumah,...,Bagus Sekali,2.0,1.0,1.0,SHM - Sertifikat Hak Milik,4400,9 Desember 2020,https://www.rumah123.com/properti/jakarta-timu...,NaN,"Cipinang, Jakarta Timur"


In [12]:
df['lokasi'] = df['lokasi'].apply(lambda x : x.split(',')[0])

In [13]:
df['lokasi'].sample(10)

5863         Permata Buana
3731               Cibubur
7730    Pantai Indah Kapuk
6511          Muara Karang
1966       Bendungan Hilir
4539            Taman Ratu
538             Johar Baru
520                 Petojo
4811              S Parman
568          Cempaka Putih
Name: lokasi, dtype: object

In [14]:
### Dictionary Kecamatan di Jakarta
kecamatan={
'Cempaka Putih' : ['Cempaka Putih Barat','Cempaka Putih Timur', 'Rawasari','Rawa Sari'],
'Gambir' : ['Cideng','Duri Pulo', 'Gambir', 'Kebon Kelapa', 'Petojo Selatan', 'Petojo Utara',
           'Roxy','Petojo'],
'Johar Baru' : ['Galur', 'Johar Baru', 'Kampung Rawa', 'Tanah Tinggi'],
'Kemayoran' : ['Cempaka Baru', 'Gunung Sahari Selatan', 'Harapan Mulya', 'Kebon Kosong',
               'Kemayoran', 'Serdang', 'Sumur Batu', 'Utan Panjang'],
'Menteng' : ['Cikini', 'Gondangdia', 'Kebon Sirih', 'Menteng', 'Pegangsaan'],
'Sawah Besar': ['Gunung Sahari Utara', 'Karang Anyar', 'Kartini', 'Kampung Anyar',
                'Mangga Dua Selatan', 'Pasar Baru', 'Gunung Sahari'],
'Senen' : ['Bungur', 'Kenari', 'Kramat', 'Kwitang', 'Paseban', 'Senen'],
'Tanah Abang' : ['Bendungan Hilir', 'Gelora', 'Kampung Bali', 'Karet Tengsin', 
                 'Kebon Kacang', 'Kebon Melati', 'Petamburan', 'KH Mas Mansyur',
                'Kebon Kacang Lama'],
'Cilincing' : ['Cilincing', 'Kalibaru', 'Marunda', 'Rorotan','Suka Pura', 'Semper',
                'Semper Barat', 'Semper Timur', 'Sukapura'],
'Kelapa Gading' : ['Kelapa Gading Barat', 'Kelapa Gading Timur', 'Pegangsaan Dua'],
'Koja' : ['Koja', 'Lagoa', 'Rawa Badak Selatan', 'Rawa Badak Utara','Tugu',
          'Tugu Selatan', 'Tugu Utara','Rawabadak Selatan'],
'Pademangan' : ['Ancol', 'Pademangan Barat', 'Pademangan Timur'],
'Penjaringan' : ['Kamal Muara', 'Kapuk Muara', 'Pejagalan', 'Penjaringan', 'Pluit',
                'Pantai Indah Kapuk','Bandengan','Teluk Gong','Muara Karang'],
'Tanjung Priok' : ['Kebon Bawang', 'Papanggo', 'Sungai Bambu', 'Sunter Agung', 'Sunter',
                    'Sunter Jaya', 'Tanjung Priok', 'Warakas'],
'Cakung' : ['Cakung Barat', 'Cakung Timur', 'Jatinegara', 'Penggilingan', 
            'Pulo Gebang', 'Rawa Terate', 'Ujung Menteng'],
'Cipayung' : ['Bambu Apus', 'Ceger', 'Cilangkap', 'Cipayung', 'Lubang Buaya',
              'Munjul', 'Pondok Ranggon', 'Setu'],
'Ciracas' : ['Cibubur', 'Ciracas', 'Kelapa Dua Wetan', 'Rambutan Susukan','Kampung Rambutan'],
'Duren Sawit' : ['Duren Sawit', 'Klender', 'Malaka Jaya', 'Malaka Sari', 
                 'Pondok Bambu', 'Pondok Kelapa', 'Pondok Kopi'],
'Jatinegara' :['Bali Mester', 'Bidara Cina', 'Cipinang Besar Selatan', 'Cipinang Besar Utara', 
                'Cipinang Cempedak', 'Cipinang Muara', 'Kampung Melayu', 'Rawa Bunga'],
'Kramat Jati': ['Balekambang', 'Batu Ampar', 'Cawang', 'Cililitan',
                'Dukuh', 'Kramat Jati', 'Tengah', 'Condet'],
'Makasar' : ['Cipinang Melayu', 'Halim Perdana Kusuma', 'Kebon Pala', 'Makasar', 
             'Pinang Ranti','Halim Perdana Kusumah'],
'Matraman' : ['Kayu Manis', 'Kebon Manggis', 'Pal Meriam', 'Pisangan Baru', 
              'Utan Kayu Selatan', 'Utan Kayu Utara','Utan Kayu'],
'Pasar Rebo' : ['Baru', 'Cijantung', 'Gedong', 'Kalisari', 'Pekayon'],
'Pulo Gadung' : ['Cipinang', 'Jati', 'Jatinegara Kaum', 'Kayu Putih', 
                  'Pisangan Timur', 'Pulo Gadung', 'Rawamangun'],
'Cilandak' : ['Cilandak Barat', 'Cipete Selatan', 'Gandaria Selatan', 'TB Simatupang', 'Cipete',
              'Lebak Bulus', 'Pondok Labu', 'Fatmawati'],
'Jagakarsa' :  ['Ciganjur', 'Cipedak', 'Jagakarsa', 'Lenteng Agung', 
                'Srengseng Sawah', 'Tanjung Barat'],
'Kebayoran Baru' : ['Cipete Utara', 'Gandaria Utara', 'Gunung', 'Kramat Pela', 'Radio Dalam',
                    'Melawai', 'Petogogan', 'Pulo', 'Rawa Barat', 'Selong', 'Senayan',
                   'Antasari','Panglima Polim','Gandaria'],
'Kebayoran Lama' : ['Cipulir', 'Grogol Selatan', 'Grogol Utara', 'Tanah Kusir', 'Permata Hijau',
                    'Kebayoran Lama Selatan', 'Kebayoran Lama Utara', 'Pondok Pinang',
                   'Pondok Indah'],
'Mampang Prapatan' : ['Bangka', 'Kuningan Barat', 'Mampang Prapatan','Kemang',
                      'Pela Mampang', 'Tegal Parang','Warung Buncit'],
'Pancoran' : ['Cikoko', 'Duren Tiga', 'Kalibata', 'Pancoran', 'Pengadegan', 'Rawajati'],
'Pasar Minggu' : ['Cilandak Timur', 'Jati Padang', 'Kebagusan', 'Pejaten', 'Ampera',
                  'Pasar Minggu', 'Pejaten Barat', 'Pejaten Timur', 'Ragunan'],
'Pesanggrahan' : ['Bintaro', 'Pesanggrahan', 'Petukangan Selatan', 'Petukangan',
                  'Petukangan Utara', 'Ulujami'],
'Setiabudi' : ['Guntur', 'Karet Kuningan', 'Karet Semanggi', 'Karet', 'Kuningan Timur',
               'Menteng Atas', 'Pasar Manggis', 'Setiabudi','Kuningan','SetiaBudi'],
'Tebet' : ['Bukit Duri', 'Kebon Baru', 'Manggarai Selatan', 'Manggarai', 'MT Haryono',
           'Menteng Dalam', 'Tebet Barat', 'Tebet Timur','Casablanca'],
'Cengkareng' : ['Cengkareng Barat', 'Cengkareng Timur', 'Duri Kosambi', 
                'Kapuk', 'Kedaung Kali Angke', 'Rawa Buaya','Daan Mogot'],
'Grogol Petamburan' : ['Grogol', 'Jelambar Baru', 'Jelambar', 'Tanjung Duren Selatan', 
                       'Tanjung Duren Utara', 'Tomang', 'Wijaya Kusuma', 'Tanjung Duren',
                      'Wijaya Kesuma'],
'Taman Sari' : ['Glodok', 'Keagungan', 'Krukut', 'Mangga Besar', 'Maphar','Kota', 
                'Pinangsia', 'Taman Sari', 'Tangki','Hayam Wuruk','Gajah Mada'],
'Tambora' : ['Angke', 'Duri Selatan', 'Duri Utara', 'Jembatan Besi', 'Jembatan Lima',
             'Kali Anyar', 'Krendang', 'Pekojan', 'Roa Malaka', 'Tambora', 'Tanah Sereal'],
'Kebon Jeruk' : ['Duri Kepa', 'Kebon Jeruk', 'Kedoya Selatan', 'Kedoya Utara','Sunrise Garden',
                 'Kelapa Dua', 'Sukabumi Selatan', 'Sukabumi Utara','Pos Pengumben',
                'Kedoya'],
'Kalideres' : ['Kalideres', 'Kamal', 'Pegadungan', 'Semanan', 'Tegal Alur'],
'Palmerah' : ['Jatipulo', 'Kemanggisan', 'Kota Bambu Selatan', 'Kota Bambu','Jati Pulo',
              'Kota Bambu Utara', 'Palmerah', 'Slipi'],
'Kembangan' : ['Joglo', 'Kembangan Selatan', 'Kembangan Utara', 'Puri Indah',
               'Meruya Selatan', 'Meruya Utara', 'Srengseng','Meruya']
}

In [15]:
## Fungsi Lokasi ke Kecamatan
def kcmtn(lokasi):
    hasil = lokasi+'_unknown'
    for i in kecamatan.keys():
        if lokasi.lower() != i.lower():
            for j in kecamatan[i]:
                if lokasi.lower() == j.lower():
                    hasil = i
                    break
        else:
            hasil = i
            break
    return hasil

In [16]:
## Apply function kcmtn to df
df['lokasi'] = df['lokasi'].apply(lambda x : kcmtn(x))
df['lokasi']

0               Senen
1       Cempaka Putih
2           Kemayoran
3               Senen
4       Cempaka Putih
            ...      
9863        Jagakarsa
9864        Jagakarsa
9865        Jagakarsa
9866        Jagakarsa
9867        Jagakarsa
Name: lokasi, Length: 9681, dtype: object

In [17]:
## Find unmapped location
counter = 0
unmapped_loc = []
for i in range(len(df)):
    if 'unknown' in df.iloc[i]['lokasi']:
        counter+=1
        loc = df.iloc[i]['lokasi'].split('_')[0]
        unmapped_loc.append((i,loc))
print(counter)
unmapped_loc[:10]

1390


[(57, 'CBD Area'),
 (150, 'Salemba'),
 (161, 'Salemba'),
 (163, 'Cempaka Mas'),
 (168, 'Percetakan Negara'),
 (182, 'Salemba'),
 (185, 'Pejompongan'),
 (187, 'Thamrin'),
 (192, 'Thamrin'),
 (214, 'Salemba')]

In [18]:
### Find unique value in unmapped location
loc = []
for i in unmapped_loc:
    loc.append(i[1])

loc_set=set(loc)
print(len(loc_set))
loc_set

96


{'Alfa Indah',
 'Bandara',
 'Batu Ceper',
 'Batutulis',
 'Blok M',
 'Bojong Indah',
 'Buaran',
 'CBD Area',
 'Cassablanca',
 'Cempaka Mas',
 'Central Park',
 'Cinere',
 'Cirendeu',
 'Citra Garden',
 'Citra Grand',
 'Dewi Sartika',
 'Gatot Subroto',
 'Gelong',
 'Gelong Baru',
 'Green Lake City',
 'Green Mansion',
 'Green Ville',
 'Green garden',
 'Gudang Peluru',
 'Harmoni',
 'Intercon',
 'Jatiwaringin',
 'Kalimalang',
 'Kampung Ambon',
 'Karang Mulia',
 'Kav DKI',
 'Kedoya Baru',
 'Kedoya Garden',
 'Kemandoran',
 'Kembangan Baru',
 'Kepa Duri',
 'Kota Wisata',
 'Legenda Wisata',
 'Mampang',
 'Mega Kuningan',
 'Metland Puri',
 'Metro permata',
 'Otista',
 'Pakubuwono',
 'Pangeran Jayakarta',
 'Pantai Mutiara',
 'Patal Senayan',
 'Pejompongan',
 'Percetakan Negara',
 'Permata Buana',
 'Pisangan Lama',
 'Plumpang',
 'Pondok Gede',
 'Prapanca',
 'Prima Kedoya',
 'Pulo Asem',
 'Pulogebang',
 'Pulomas',
 'Puri Mansion',
 'Puri Media',
 'Raffles Hills',
 'Rawa Badak',
 'Rawa Belong',
 'Ring R

In [19]:
## Find Value Counts for unknown location
def isunknown(lokasi):
    if 'unknown' in lokasi.lower():
        return 1
    else:
        return 0
df['unkown_loc'] = df['lokasi'].apply(lambda x : isunknown(x))

In [20]:
df.head(1)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc
0,2.500000e+10,Senen,Luas Bangunan : 1553 m²,Luas Tanah : 774 m²,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,3.0,2.0,2.0,SHM - Sertifikat Hak Milik,1300,6 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat",0


In [21]:
df_unknown = df[df['unkown_loc']==1]
df_unknown.head()

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc
57,3.500000e+09,CBD Area_unknown,Luas Bangunan : 130 m²,Luas Tanah : -,3.0,2.0,NaN,Apartemen Pavilion Sudirman Full Furnished Cit...,aps2432270,Apartemen,...,NaN,1.0,1.0,HGB - Hak Guna Bangunan,NaN,24 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,"Water Filter, Refrigerator, Washing Machine, A...","CBD Area, Jakarta Pusat",1
157,1.150000e+09,Salemba_unknown,Luas Bangunan : 55 m²,Luas Tanah : -,2.0,1.0,NaN,aparteman salemba residence jakarta pusat:Apar...,aps2325661,Apartemen,...,15.0,NaN,NaN,HGB - Hak Guna Bangunan,NaN,4 September 2020,https://www.rumah123.com/properti/jakarta-pusa...,"Refrigerator, Stove, AC, Water Heater","Salemba, Jakarta Pusat",1
169,1.245000e+10,Salemba_unknown,Luas Bangunan : -,Luas Tanah : 450 m²,5.0,3.0,8.0,Dijual Cepat Rumah dng Paviliun di Salemba Ten...,hos1602665,Rumah,...,NaN,NaN,NaN,"Lainnya (PPJB,Girik,Adat,dll)",NaN,20 Mei 2015,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Salemba, Jakarta Pusat",1
172,2.700000e+09,Cempaka Mas_unknown,Luas Bangunan : 100 m²,Luas Tanah : 168 m²,5.0,3.0,1.0,Rumah Murah di Sumur Batu 16 Juta/m. Pasaran d...,hos8283066,Rumah,...,2.0,NaN,NaN,SHM - Sertifikat Hak Milik,1300,10 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Mas, Jakarta Pusat",1
177,4.100000e+08,Percetakan Negara_unknown,Luas Bangunan : 34 m²,Luas Tanah : -,2.0,NaN,NaN,Apartemen the green pramuka:GREEN PROPERTY MAN...,aps1495785,Apartemen,...,7.0,NaN,NaN,HGB - Hak Guna Bangunan,NaN,4 Maret 2016,https://www.rumah123.com/properti/jakarta-pusa...,"Carport, Swimming Pool, Garden","Percetakan Negara, Jakarta Pusat",1


In [22]:
df_unknown['lokasi'].value_counts()

Green Lake City_unknown    118
Citra Garden_unknown        99
Taman Ratu_unknown          94
Permata Buana_unknown       73
Citra Grand_unknown         65
                          ... 
Karang Mulia_unknown         1
Plumpang_unknown             1
Kemandoran_unknown           1
Villa Meruya_unknown         1
Patal Senayan_unknown        1
Name: lokasi, Length: 96, dtype: int64

In [23]:
# print(df_unknown['lokasi'].value_counts().to_string())

In [24]:
loc_dict = dict.fromkeys(loc_set, 0)

In [25]:
loc_dict['Green Lake City'] = 'Cengkareng'
loc_dict['Citra Garden'] = 'Kalideres'
loc_dict['Taman Ratu'] = 'Kebon Jeruk'
loc_dict['Permata Buana'] = 'Kembangan'
loc_dict['Citra Grand'] = 'Ciracas'
loc_dict['Pantai Mutiara'] = 'Penjaringan'
loc_dict['Thamrin'] = 'Menteng'
loc_dict['Kota Wisata'] = 'Cipayung'
loc_dict['Pulomas'] = 'Pulo Gadung'
loc_dict['Taman Anggrek'] = 'Grogol Petamburan'
loc_dict['Green garden'] = 'Kebon Jeruk'
loc_dict['Intercon'] = 'Kebon Jeruk'
loc_dict['Pulogebang'] = 'Pulo Gadung'
loc_dict['Green Ville'] = 'Kebon Jeruk'
loc_dict['Puri Mansion'] = 'Kembangan'
loc_dict['Percetakan Negara'] = 'Cempaka Putih'
loc_dict['Taman Palem'] = 'Kalideres'
loc_dict['Salemba'] = 'Senen'
# loc_dict['Metland Puri'] = '' ======> Tangerang
loc_dict['Jatiwaringin'] = 'Makasar'
loc_dict['Bojong Indah'] = 'Cengkareng'
loc_dict['Cempaka Mas'] = 'Kemayoran'
loc_dict['Buaran'] = 'Duren Sawit'
loc_dict['Kav DKI'] = 'Kembangan'
loc_dict['Cirendeu'] = 'Cilandak'
loc_dict['Central Park'] = 'Grogol Petamburan'
loc_dict['Metro permata'] = 'Kalideres'
loc_dict['Pulo Asem'] = 'Pulo Gadung'
loc_dict['Senopati'] = 'Kebayoran Baru'
loc_dict['Legenda Wisata'] = 'Cipayung'
loc_dict['Kepa Duri'] = 'Kebon Jeruk'
loc_dict['Kedoya Baru'] = 'Kebon Jeruk'
loc_dict['Mega Kuningan'] = 'Setiabudi'
loc_dict['Gelong'] = 'Grogol Petamburan'
loc_dict['Taman Surya'] = 'Kalideres'
loc_dict['Otista'] = 'Jatinegara'
loc_dict['Pejompongan'] = 'Tanah Abang'
# loc_dict['Cinere'] = '' ==============> Depok
loc_dict['Cassablanca'] = 'Setiabudi'
loc_dict['Pakubuwono'] = 'Kebayoran Baru'
loc_dict['Kampung Ambon'] = 'Pulo Gadung'
loc_dict['Taman Grisenda'] = 'Penjaringan'
loc_dict['patra kuningan'] = 'Setiabudi'
loc_dict['Veteran'] = 'Pesanggrahan'
loc_dict['Mampang'] = 'Mampang Prapatan'
loc_dict['Taman Mini'] = 'Cipayung'
loc_dict['Batu Ceper'] = 'Gambir'
loc_dict['Taman Cosmos'] = 'Kebon Jeruk'
loc_dict['S Parman'] = 'Grogol Petamburan'
loc_dict['Puri Media'] = 'Kembangan'
loc_dict['Alfa Indah'] = 'Pesanggrahan'
loc_dict['Simprug'] = 'Kebayoran Lama'
loc_dict['Taman Meruya'] = 'Kembangan'
loc_dict['Prapanca'] = 'Kebayoran Baru'
loc_dict['SCBD'] = 'Kebayoran Baru'
loc_dict['Simprug Garden'] = 'Kebayoran Lama'
loc_dict['Pondok Gede'] = 'Makasar'
loc_dict['Pangeran Jayakarta'] = 'Sawah Besar'
loc_dict['Gelong Baru'] = 'Grogol Petamburan'
loc_dict['Green Mansion'] = 'Cengkareng'
loc_dict['Dewi Sartika'] = 'Kramat Jati'
loc_dict['Sektor 7-Bintaro'] = 'Pesanggrahan'
loc_dict['Harmoni'] = 'Kebayoran Baru'
loc_dict['Tamansari'] = 'Taman Sari'
loc_dict['Gudang Peluru'] = 'Tebet'
loc_dict['Gatot Subroto'] = 'Tanah Abang'
loc_dict['Rawa Badak'] = 'Koja'
loc_dict['Tubagus Angke'] = 'Tambora'
loc_dict['Pisangan Lama'] = 'Pulo Gadung'
loc_dict['CBD Area'] = 'Kebayoran Baru'
loc_dict['Raffles Hills'] = 'Ciracas'
loc_dict['Ring Road'] = 'Cengkareng'
loc_dict['Saharjo'] = 'Tebet'
loc_dict['Terogong'] = 'Cilandak'
loc_dict['Blok M'] = 'Kebayoran Baru'
loc_dict['Sektor 1 - Bintaro'] = 'Pesanggrahan'
loc_dict['Taman Kencana'] = 'Cengkareng'
loc_dict['Samanhudi'] = 'Sawah Besar'
loc_dict['Sektor 2 - Bintaro'] = 'Pesanggrahan'

loc_dict

{'Harmoni': 'Kebayoran Baru',
 'Gelong': 'Grogol Petamburan',
 'Pisangan Lama': 'Pulo Gadung',
 'Buaran': 'Duren Sawit',
 'Kemandoran': 0,
 'Kedoya Baru': 'Kebon Jeruk',
 'Cinere': 0,
 'Blok M': 'Kebayoran Baru',
 'Raffles Hills': 'Ciracas',
 'Green garden': 'Kebon Jeruk',
 'Pantai Mutiara': 'Penjaringan',
 'Prima Kedoya': 0,
 'Sektor 1 - Bintaro': 'Pesanggrahan',
 'Pakubuwono': 'Kebayoran Baru',
 'Rawa Badak': 'Koja',
 'Samanhudi': 'Sawah Besar',
 'Green Mansion': 'Cengkareng',
 'CBD Area': 'Kebayoran Baru',
 'Villa Meruya': 0,
 'Pondok Gede': 'Makasar',
 'Taman Anggrek': 'Grogol Petamburan',
 'Bandara': 0,
 'Rawa Belong': 0,
 'Pulogebang': 'Pulo Gadung',
 'Pejompongan': 'Tanah Abang',
 'Sektor 7-Bintaro': 'Pesanggrahan',
 'Simprug Garden': 'Kebayoran Lama',
 'Puri Media': 'Kembangan',
 'Kalimalang': 0,
 'SCBD': 'Kebayoran Baru',
 'Sudirman': 0,
 'Legenda Wisata': 'Cipayung',
 'Taman Palem': 'Kalideres',
 'Central Park': 'Grogol Petamburan',
 'Kav DKI': 'Kembangan',
 'Taman Grisenda':

In [27]:
# df.head()
df[df['unkown_loc'] == 1]
# for i in range(len(df)):
#     if 'unknown' in df['lokasi'].iloc[i]:
#         df['lokasi'].iloc[i] = df['lokasi'].iloc[i].split('_')[0]
# df[df['unkown_loc'] == 1]

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc
57,3.500000e+09,CBD Area_unknown,Luas Bangunan : 130 m²,Luas Tanah : -,3.0,2.0,NaN,Apartemen Pavilion Sudirman Full Furnished Cit...,aps2432270,Apartemen,...,NaN,1.0,1.0,HGB - Hak Guna Bangunan,NaN,24 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,"Water Filter, Refrigerator, Washing Machine, A...","CBD Area, Jakarta Pusat",1
157,1.150000e+09,Salemba_unknown,Luas Bangunan : 55 m²,Luas Tanah : -,2.0,1.0,NaN,aparteman salemba residence jakarta pusat:Apar...,aps2325661,Apartemen,...,15.0,NaN,NaN,HGB - Hak Guna Bangunan,NaN,4 September 2020,https://www.rumah123.com/properti/jakarta-pusa...,"Refrigerator, Stove, AC, Water Heater","Salemba, Jakarta Pusat",1
169,1.245000e+10,Salemba_unknown,Luas Bangunan : -,Luas Tanah : 450 m²,5.0,3.0,8.0,Dijual Cepat Rumah dng Paviliun di Salemba Ten...,hos1602665,Rumah,...,NaN,NaN,NaN,"Lainnya (PPJB,Girik,Adat,dll)",NaN,20 Mei 2015,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Salemba, Jakarta Pusat",1
172,2.700000e+09,Cempaka Mas_unknown,Luas Bangunan : 100 m²,Luas Tanah : 168 m²,5.0,3.0,1.0,Rumah Murah di Sumur Batu 16 Juta/m. Pasaran d...,hos8283066,Rumah,...,2.0,NaN,NaN,SHM - Sertifikat Hak Milik,1300,10 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Mas, Jakarta Pusat",1
177,4.100000e+08,Percetakan Negara_unknown,Luas Bangunan : 34 m²,Luas Tanah : -,2.0,NaN,NaN,Apartemen the green pramuka:GREEN PROPERTY MAN...,aps1495785,Apartemen,...,7.0,NaN,NaN,HGB - Hak Guna Bangunan,NaN,4 Maret 2016,https://www.rumah123.com/properti/jakarta-pusa...,"Carport, Swimming Pool, Garden","Percetakan Negara, Jakarta Pusat",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9673,5.000000e+10,patra kuningan_unknown,Luas Bangunan : 800 m²,Luas Tanah : 650 m²,5.0,5.0,3.0,Dijual Rumah Mewah Siap Huni di Patra Kuningan...,hos8147864,Rumah,...,NaN,NaN,NaN,NaN,NaN,9 April 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"patra kuningan, Jakarta Selatan",1
9678,8.250000e+09,Gudang Peluru_unknown,Luas Bangunan : 400 m²,Luas Tanah : 393 m²,5.0,3.0,3.0,"Rumah Siap Huni, Dalam Kompleks, Tenang & Aman...",hos8176388,Rumah,...,NaN,NaN,NaN,NaN,NaN,15 April 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Gudang Peluru, Jakarta Selatan",1
9690,2.500000e+09,Terogong_unknown,Luas Bangunan : 97 m²,Luas Tanah : -,2.0,2.0,NaN,"For Sale/Rent 2BR+1 @ Hampton Apartment, Terog...",aps2384309,Apartemen,...,NaN,NaN,NaN,NaN,NaN,11 Januari 2021,https://www.rumah123.com/properti/jakarta-sela...,"Air Conditioning, Swimming Pool","Terogong, Jakarta Selatan",1
9701,3.500000e+09,Cassablanca_unknown,Luas Bangunan : 153 m²,Luas Tanah : -,4.0,2.0,NaN,Apartemen Casablanca Mansion Full Furnished ha...,aps1990611,Apartemen,...,NaN,1.0,1.0,HGB - Hak Guna Bangunan,NaN,30 Oktober 2019,https://www.rumah123.com/properti/jakarta-sela...,"AC, Swimming Pool","Cassablanca, Jakarta Selatan",1


In [28]:
for i in range(len(df)):
    if 'unknown' in df['lokasi'].iloc[i]:
        df['lokasi'].iloc[i] = loc_dict[df['lokasi'].iloc[i].split('_')[0]]

In [33]:
counter = 0
for i in range(len(df)):
    if df['lokasi'].iloc[i] in kecamatan.keys():
        pass
    else:
        counter+=1
        print(i,df.iloc[i]['lokasi_copy'])
counter

299 Batutulis, Jakarta Pusat
3638 Kalimalang, Jakarta Timur
4144 Villa Meruya, Jakarta Barat
4193 Metland Puri, Jakarta Barat
4194 Metland Puri, Jakarta Barat
4195 Metland Puri, Jakarta Barat
4256 Metland Puri, Jakarta Barat
4286 Karang Mulia, Jakarta Barat
4478 Metland Puri, Jakarta Barat
4674 Metland Puri, Jakarta Barat
4728 Metland Puri, Jakarta Barat
4862 Metland Puri, Jakarta Barat
4952 Metland Puri, Jakarta Barat
4967 Metland Puri, Jakarta Barat
4988 Metland Puri, Jakarta Barat
5034 Metland Puri, Jakarta Barat
5057 Kembangan Baru, Jakarta Barat
5140 Bandara, Jakarta Barat
5284 Metland Puri, Jakarta Barat
5290 Metland Puri, Jakarta Barat
5291 Kedoya Garden, Jakarta Barat
5310 Metland Puri, Jakarta Barat
5329 Prima Kedoya, Jakarta Barat
5366 Metland Puri, Jakarta Barat
5367 Metland Puri, Jakarta Barat
5378 Metland Puri, Jakarta Barat
5387 Metland Puri, Jakarta Barat
5478 Rawa Belong, Jakarta Barat
5593 Tanjung Gedong, Jakarta Barat
7407 Plumpang, Jakarta Utara
7869 Cinere, Jakarta 

45

In [34]:
df[df['lokasi']==0]

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc
327,5.000000e+07,0,Luas Bangunan : 558 m²,Luas Tanah : 558 m²,NaN,NaN,NaN,"Rumah Batu Tulis Raya:Rumah Batu Tulis Raya, l...",hos8286340,Rumah,...,NaN,NaN,NaN,SHM - Sertifikat Hak Milik,NaN,11 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Batutulis, Jakarta Pusat",1
3825,1.950000e+10,0,Luas Bangunan : 733 m²,Luas Tanah : 1324 m²,4.0,3.0,NaN,Rumah Tua Harus Renovasi Lokasi Jalan Raya:Dij...,hos8276407,Rumah,...,1.0,NaN,NaN,SHM - Sertifikat Hak Milik,NaN,8 Mei 2021,https://www.rumah123.com/properti/jakarta-timu...,NaN,"Kalimalang, Jakarta Timur",1
4331,9.800000e+09,0,Luas Bangunan : 740 m²,Luas Tanah : 350 m²,5.0,5.0,NaN,"Rumah Taman Villa Meruya , Luas tanah 350 m2 ....",hos8210295,Rumah,...,NaN,NaN,NaN,SHM - Sertifikat Hak Milik,16500,22 April 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Villa Meruya, Jakarta Barat",1
4380,3.200000e+09,0,Luas Bangunan : 131 m²,Luas Tanah : 120 m²,3.0,3.0,NaN,"Rumah Riviera Puri , phase 1 , ukuran 8x15 , h...",hos7484071,Rumah,...,NaN,1.0,1.0,"Lainnya (PPJB,Girik,Adat,dll)",NaN,22 April 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Metland Puri, Jakarta Barat",1
4381,5.500000e+09,0,Luas Bangunan : 211 m²,Luas Tanah : 180 m²,4.0,4.0,NaN,"Rumah Riviera Puri , Phase 1 , ukuran 10x18m2 ...",hos7484110,Rumah,...,NaN,4.0,4.0,"Lainnya (PPJB,Girik,Adat,dll)",NaN,22 April 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Metland Puri, Jakarta Barat",1
4382,4.400000e+09,0,Luas Bangunan : 164 m²,Luas Tanah : 144 m²,3.0,3.0,NaN,"Rumah Riviera puri , phase 3 , ukuran 8x18m2 ,...",hos7484097,Rumah,...,NaN,1.0,1.0,"Lainnya (PPJB,Girik,Adat,dll)",NaN,22 April 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Metland Puri, Jakarta Barat",1
4443,2.200000e+09,0,Luas Bangunan : 120 m²,Luas Tanah : 120 m²,3.0,3.0,2.0,rumah metland puri..baru..jln 2 mobil..jakarta...,hos8027283,Rumah,...,2.0,1.0,1.0,SHM - Sertifikat Hak Milik,2200,12 Maret 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Metland Puri, Jakarta Barat",1
4473,1.200000e+09,0,Luas Bangunan : 100 m²,Luas Tanah : 54 m²,2.0,3.0,1.0,"Frans cen-10 Menit dari Puri Indah, Permata Bu...",hos7661558,Rumah,...,2.0,1.0,1.0,SHM - Sertifikat Hak Milik,2200,11 Desember 2020,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Karang Mulia, Jakarta Barat",1
4665,2.600000e+09,0,Luas Bangunan : 180 m²,Luas Tanah : 160 m²,NaN,NaN,NaN,Rumah Murah Metland Puri. Tangerang:lokasi str...,hos8287268,Rumah,...,1.0,NaN,NaN,SHM - Sertifikat Hak Milik,NaN,11 Mei 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Metland Puri, Jakarta Barat",1
4861,2.800000e+09,0,Luas Bangunan : 172 m²,Luas Tanah : 160 m²,4.0,4.0,2.0,Rumah 2 lt di Metland Puri Tanggerang Banten:L...,hos8101273,Rumah,...,2.0,1.0,NaN,SHM - Sertifikat Hak Milik,4400,30 Maret 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Metland Puri, Jakarta Barat",1


In [35]:
df.drop(df[df['lokasi'] == 0].index, inplace = True)

In [36]:
counter = 0
for i in range(len(df)):
    if df['lokasi'].iloc[i] in kecamatan.keys():
        pass
    else:
        counter+=1
        print(i,df.iloc[i]['lokasi_copy'])
counter

0

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9636 entries, 0 to 9867
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9636 non-null   float64
 1   lokasi                9636 non-null   object 
 2   lb                    9636 non-null   object 
 3   lt                    9636 non-null   object 
 4   kt                    9356 non-null   float64
 5   km                    9112 non-null   float64
 6   car                   3974 non-null   float64
 7   Deskripsi             9636 non-null   object 
 8   ID Iklan              9636 non-null   object 
 9   Tipe Properti         9636 non-null   object 
 10  Dilengkapi Perabotan  5718 non-null   object 
 11  Kondisi Properti      6543 non-null   object 
 12  Jumlah Lantai         7070 non-null   float64
 13  Kamar Pembantu        4128 non-null   float64
 14  Kamar Mandi Pembantu  3739 non-null   float64
 15  Sertifikat           

In [40]:
df.drop(columns=['unkown_loc'])

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Kondisi Properti,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy
0,2.500000e+10,Senen,Luas Bangunan : 1553 m²,Luas Tanah : 774 m²,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,Bagus Sekali,3.0,2.0,2.0,SHM - Sertifikat Hak Milik,1300,6 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat"
1,1.750000e+09,Cempaka Putih,Luas Bangunan : 97 m²,Luas Tanah : 49 m²,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,Bagus Sekali,3.0,NaN,NaN,SHM - Sertifikat Hak Milik,2200,27 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Putih, Jakarta Pusat"
2,1.500000e+09,Kemayoran,Luas Bangunan : 110 m²,Luas Tanah : 70 m²,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,...,Bagus,2.0,1.0,1.0,SHM - Sertifikat Hak Milik,2200,10 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Sumur Batu, Jakarta Pusat"
3,1.750000e+09,Senen,Luas Bangunan : 150 m²,Luas Tanah : 84 m²,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,...,NaN,2.0,NaN,NaN,"Lainnya (PPJB,Girik,Adat,dll)",NaN,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat"
4,1.375000e+09,Cempaka Putih,Luas Bangunan : 65 m²,Luas Tanah : 35 m²,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,...,Baru,2.0,NaN,NaN,SHM - Sertifikat Hak Milik,2200,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Putih, Jakarta Pusat"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9863,1.100000e+09,Jagakarsa,Luas Bangunan : 100 m²,Luas Tanah : 63 m²,3.0,2.0,1.0,Hanya 5 Unit siapa cepat dia dapat:''Hanya 5 u...,hos7846972,Rumah,...,Baru,2.0,NaN,NaN,SHM - Sertifikat Hak Milik,2200,29 Januari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan"
9864,1.350000e+09,Jagakarsa,Luas Bangunan : 110 m²,Luas Tanah : 95 m²,3.0,3.0,1.0,Rumah Cluster Indent siap Bangun:Hunian Cluste...,hos7878697,Rumah,...,Baru,2.0,1.0,1.0,SHM - Sertifikat Hak Milik,2200,5 Februari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan"
9865,1.550000e+09,Jagakarsa,Luas Bangunan : 130 m²,Luas Tanah : 95 m²,3.0,3.0,2.0,Cluster exclusive Best area kebagusan:*Hunian ...,hos7875576,Rumah,...,Baru,2.0,1.0,NaN,SHM - Sertifikat Hak Milik,1300,4 Februari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan"
9866,1.350000e+09,Jagakarsa,Luas Bangunan : 130 m²,Luas Tanah : 86 m²,3.0,3.0,1.0,Rumah Cluster Jakarta selatan jagakarsa:Hunian...,hos7861060,Rumah,...,Baru,2.0,1.0,NaN,SHM - Sertifikat Hak Milik,1300,2 Februari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan"


### Munging kolom lb dan lt

In [41]:
df['lb'].sample(20)

5720     Luas Bangunan : 120 m²
1334    Luas Bangunan : 1553 m²
8986     Luas Bangunan : 276 m²
1781     Luas Bangunan : 205 m²
7730          Luas Bangunan : -
3254      Luas Bangunan : 60 m²
5231     Luas Bangunan : 400 m²
2481     Luas Bangunan : 350 m²
1948      Luas Bangunan : 86 m²
5535     Luas Bangunan : 941 m²
674       Luas Bangunan : 27 m²
5455     Luas Bangunan : 160 m²
3964     Luas Bangunan : 175 m²
7068      Luas Bangunan : 25 m²
6692     Luas Bangunan : 450 m²
9398     Luas Bangunan : 350 m²
2629     Luas Bangunan : 300 m²
4145     Luas Bangunan : 300 m²
615       Luas Bangunan : 34 m²
5105     Luas Bangunan : 192 m²
Name: lb, dtype: object

In [42]:
df['lt'].sample(20)

8949    Luas Tanah : 350 m²
8765         Luas Tanah : -
7540    Luas Tanah : 150 m²
9161         Luas Tanah : -
8220    Luas Tanah : 252 m²
2051     Luas Tanah : 60 m²
3397    Luas Tanah : 222 m²
1976     Luas Tanah : 45 m²
176     Luas Tanah : 774 m²
655     Luas Tanah : 162 m²
8166    Luas Tanah : 412 m²
2948     Luas Tanah : 90 m²
3627    Luas Tanah : 180 m²
6636    Luas Tanah : 720 m²
5126     Luas Tanah : 75 m²
5287    Luas Tanah : 126 m²
3685     Luas Tanah : 80 m²
6468    Luas Tanah : 150 m²
7509     Luas Tanah : 75 m²
9045    Luas Tanah : 440 m²
Name: lt, dtype: object

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9636 entries, 0 to 9867
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9636 non-null   float64
 1   lokasi                9636 non-null   object 
 2   lb                    9636 non-null   object 
 3   lt                    9636 non-null   object 
 4   kt                    9356 non-null   float64
 5   km                    9112 non-null   float64
 6   car                   3974 non-null   float64
 7   Deskripsi             9636 non-null   object 
 8   ID Iklan              9636 non-null   object 
 9   Tipe Properti         9636 non-null   object 
 10  Dilengkapi Perabotan  5718 non-null   object 
 11  Kondisi Properti      6543 non-null   object 
 12  Jumlah Lantai         7070 non-null   float64
 13  Kamar Pembantu        4128 non-null   float64
 14  Kamar Mandi Pembantu  3739 non-null   float64
 15  Sertifikat           

In [44]:
def ModifiedAmbilAngka(text):
    try:
        angka = ''.join(re.findall(r'[^\D]+',text))
        if angka == '':
            return np.nan
        else:
            return angka
    except:
        return np.nan
df['lt'] = df['lt'].apply(lambda x : ModifiedAmbilAngka(x))
df['lb'] = df['lb'].apply(lambda x : ModifiedAmbilAngka(x))
df.head(2)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Jumlah Lantai,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc
0,2.500000e+10,Senen,1553,774,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,3.0,2.0,2.0,SHM - Sertifikat Hak Milik,1300,6 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat",0
1,1.750000e+09,Cempaka Putih,97,49,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,3.0,NaN,NaN,SHM - Sertifikat Hak Milik,2200,27 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Putih, Jakarta Pusat",0


In [45]:
# dg = pd.read_csv('data_raw_rumah123.csv')
# for i in range(len(dg)):
#     if '.' in dg.iloc[i]['lb'] or ',' in dg.iloc[i]['lb']:
#         print(dg.iloc[i]['lb'])

In [46]:
df['lb'] = df['lb'].astype(float)
df['lt'] = df['lt'].astype(float)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9636 entries, 0 to 9867
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9636 non-null   float64
 1   lokasi                9636 non-null   object 
 2   lb                    9302 non-null   float64
 3   lt                    8013 non-null   float64
 4   kt                    9356 non-null   float64
 5   km                    9112 non-null   float64
 6   car                   3974 non-null   float64
 7   Deskripsi             9636 non-null   object 
 8   ID Iklan              9636 non-null   object 
 9   Tipe Properti         9636 non-null   object 
 10  Dilengkapi Perabotan  5718 non-null   object 
 11  Kondisi Properti      6543 non-null   object 
 12  Jumlah Lantai         7070 non-null   float64
 13  Kamar Pembantu        4128 non-null   float64
 14  Kamar Mandi Pembantu  3739 non-null   float64
 15  Sertifikat           

In [48]:
df[df['lb'].isna() == True]['Tipe Properti'].value_counts()

Apartemen    214
Rumah        120
Name: Tipe Properti, dtype: int64

In [49]:
df[df['lt'].isna() == True]['Tipe Properti'].value_counts()

Apartemen    1586
Rumah          37
Name: Tipe Properti, dtype: int64

In [50]:
len(df[df['Tipe Properti']=='Apartemen'])

1586

In [51]:
df[df['Tipe Properti']=='Apartemen']['lt'].unique()

array([nan])

In [52]:
df.dropna(subset=['lb'], inplace=True)

In [53]:
df[df['lt'].isna() == True]['Tipe Properti'].value_counts()

Apartemen    1372
Rumah           9
Name: Tipe Properti, dtype: int64

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9302 entries, 0 to 9867
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9302 non-null   float64
 1   lokasi                9302 non-null   object 
 2   lb                    9302 non-null   float64
 3   lt                    7921 non-null   float64
 4   kt                    9098 non-null   float64
 5   km                    8935 non-null   float64
 6   car                   3954 non-null   float64
 7   Deskripsi             9302 non-null   object 
 8   ID Iklan              9302 non-null   object 
 9   Tipe Properti         9302 non-null   object 
 10  Dilengkapi Perabotan  5690 non-null   object 
 11  Kondisi Properti      6497 non-null   object 
 12  Jumlah Lantai         6999 non-null   float64
 13  Kamar Pembantu        4097 non-null   float64
 14  Kamar Mandi Pembantu  3709 non-null   float64
 15  Sertifikat           

In [55]:
df = df[~((df['Tipe Properti']=='Rumah') & (df['lt'].isna()==True))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9293 entries, 0 to 9867
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 9293 non-null   float64
 1   lokasi                9293 non-null   object 
 2   lb                    9293 non-null   float64
 3   lt                    7921 non-null   float64
 4   kt                    9093 non-null   float64
 5   km                    8930 non-null   float64
 6   car                   3950 non-null   float64
 7   Deskripsi             9293 non-null   object 
 8   ID Iklan              9293 non-null   object 
 9   Tipe Properti         9293 non-null   object 
 10  Dilengkapi Perabotan  5689 non-null   object 
 11  Kondisi Properti      6496 non-null   object 
 12  Jumlah Lantai         6997 non-null   float64
 13  Kamar Pembantu        4097 non-null   float64
 14  Kamar Mandi Pembantu  3709 non-null   float64
 15  Sertifikat           

##### Catatan : data yang kolom `lt` bernilai nan sekarang hanya data apartemen

### Munging kolom `kt` dan `km`

In [56]:
df[df['kt'].isna()==True]['Tipe Properti'].value_counts()

Rumah        173
Apartemen     27
Name: Tipe Properti, dtype: int64

In [57]:
df[df['km'].isna()==True]['Tipe Properti'].value_counts()

Rumah        293
Apartemen     70
Name: Tipe Properti, dtype: int64

In [58]:
df[df['Tipe Properti']=='Apartemen']['kt'].unique()

array([ 2.,  1.,  3.,  4., 73., nan,  8.,  5.])

In [59]:
len(df[(df['km'].isna()==True)&(df['kt'].isna()==True)])

200

In [60]:
df[(df['km'].isna()==True)&(df['kt'].isna()==True)]['Tipe Properti'].value_counts()

Rumah        173
Apartemen     27
Name: Tipe Properti, dtype: int64

##### Pilihan Sementara didrop saja

In [61]:
df.dropna(subset=['kt'], inplace=True)
df.dropna(subset=['km'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   kt                    8930 non-null   float64
 5   km                    8930 non-null   float64
 6   car                   3950 non-null   float64
 7   Deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   Tipe Properti         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  Jumlah Lantai         6819 non-null   float64
 13  Kamar Pembantu        4080 non-null   float64
 14  Kamar Mandi Pembantu  3703 non-null   float64
 15  Sertifikat           

### Munging Kolom `car`

In [62]:
df[df['car'].isna()==True]['Tipe Properti'].value_counts()

Rumah        3678
Apartemen    1302
Name: Tipe Properti, dtype: int64

##### Harus diisi untuk rumah dan apartemen
- Untuk rumah dicari di dekripsi
- Untuk apartemen diisi angka default, misal 1

In [72]:
df['garasi'] = df['car']
df.loc[df['Tipe Properti'] == "Apartemen", "garasi"] = 1
df.loc[df['Tipe Properti'] == "Rumah", "garasi"] = df['car']
df.head(5)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc,garasi
0,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,2.0,2.0,SHM - Sertifikat Hak Milik,1300,6 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat",0,100.0
1,1.750000e+09,Cempaka Putih,97.0,49.0,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,NaN,NaN,SHM - Sertifikat Hak Milik,2200,27 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Putih, Jakarta Pusat",0,NaN
2,1.500000e+09,Kemayoran,110.0,70.0,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,...,1.0,1.0,SHM - Sertifikat Hak Milik,2200,10 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Sumur Batu, Jakarta Pusat",0,1.0
3,1.750000e+09,Senen,150.0,84.0,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,...,NaN,NaN,"Lainnya (PPJB,Girik,Adat,dll)",NaN,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat",0,NaN
4,1.375000e+09,Cempaka Putih,65.0,35.0,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,...,NaN,NaN,SHM - Sertifikat Hak Milik,2200,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Putih, Jakarta Pusat",0,NaN


In [68]:
df[df['garasi'].isna()==True]['Tipe Properti'].value_counts()

Rumah    3678
Name: Tipe Properti, dtype: int64

In [76]:
df['garasi'] = df['garasi'].fillna(-1)
df[df['garasi'].isna()==True]['Tipe Properti'].value_counts()

Series([], Name: Tipe Properti, dtype: int64)

In [84]:
df.sample(3)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Kamar Pembantu,Kamar Mandi Pembantu,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc,garasi
9678,8.250000e+09,Tebet,400.0,393.0,5.0,3.0,3.0,"Rumah Siap Huni, Dalam Kompleks, Tenang & Aman...",hos8176388,Rumah,...,NaN,NaN,NaN,NaN,15 April 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Gudang Peluru, Jakarta Selatan",1,3.0
4634,2.750000e+09,Kebon Jeruk,150.0,119.0,5.0,3.0,NaN,Rumah bagus siap uni lokasi strategis di taman...,hos8114269,Rumah,...,NaN,NaN,SHM - Sertifikat Hak Milik,2200,2 April 2021,https://www.rumah123.com/properti/jakarta-bara...,NaN,"Taman Ratu, Jakarta Barat",1,-1.0
1408,1.100000e+10,Johar Baru,300.0,400.0,7.0,4.0,NaN,Rumah siap huni di percetakan negara:Rumah Dij...,hos6983739,Rumah,...,NaN,NaN,SHM - Sertifikat Hak Milik,5500,21 Juli 2020,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Johar Baru, Jakarta Pusat",0,-1.0


In [85]:
# def garasi(df):
#     df_copy = df.copy()
#     df_copy['Deskripsi'] = df_copy['Deskripsi'].apply(lambda x:str(x).lower())
#     df_copy['car_desc'] = df_copy['Deskripsi'].apply(
#         lambda x:0 if ('tidak ada garasi' in x) else 1 if ('carport' in x) or ('garasi' in x) \
#         else 0
#     )
#     df_copy['car_spaces'] = df_copy['car_spaces'].fillna(0)
#     df_copy['ada_garasi'] = np.where((df_copy['garasi_desc'] > 0),
#                                      1,np.where(df_copy['car_spaces'] > 0 ,1 ,0)
#                                     )
    
#     return df_copy.drop('garasi_desc',axis=1)

def adagarasi(text):
    text = text.lower()
    if 'tidak ada garasi' in text or 'tidak ada carport' in text:
        return 0
    elif 'carport' in text or 'garasi' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    if (df['Tipe Properti'].iloc[i] == "Rumah") and (df['garasi'].iloc[i] == -1):
        df['garasi'].iloc[i] = adagarasi(df['Deskripsi'].iloc[i])    

In [87]:
df['garasi'].value_counts()

0.0      3472
1.0      3066
2.0      1575
4.0       304
3.0       239
6.0        86
5.0        59
10.0       49
8.0        24
7.0        16
9.0         9
15.0        8
12.0        6
30.0        3
14.0        3
100.0       2
20.0        2
106.0       1
16.0        1
24.0        1
17.0        1
11.0        1
13.0        1
19.0        1
Name: garasi, dtype: int64

### Munging kolom `Jumlah Lantai`

In [89]:
## Normalize jumlah lantai untuk apartemen : diisi 1
df['Jumlah Lantai_copy'] = df['Jumlah Lantai']
df.loc[df['Tipe Properti'] == "Apartemen", "Jumlah Lantai"] = "1"
df[df['Tipe Properti']=='Apartemen']['Jumlah Lantai'].value_counts()

1    1302
Name: Jumlah Lantai, dtype: int64

In [90]:
df[df['Jumlah Lantai'].isna()==True]['Tipe Properti'].value_counts()

Rumah    1405
Name: Tipe Properti, dtype: int64

In [91]:
## Percobaan Mengisi Jumlah Lantai Menggunakan Ceil
ddummy = df.copy()
Ceil = ddummy[ddummy['Tipe Properti'] == 'Rumah']
Ceil = Ceil.dropna(subset=['Jumlah Lantai', 'lb', 'lt'])
Ceil = Ceil[['lb','lt','Jumlah Lantai']]
# Ceil = Ceil.astype(int)
Ceil['ceil'] = Ceil['lb']/Ceil['lt']
Ceil['ceil'] = Ceil['ceil'].apply(lambda x : math.ceil(x))
x = len(Ceil)
y = len(Ceil[Ceil['Jumlah Lantai'] != Ceil['ceil']])
akurasi = (x-y)/x
print(x,y,akurasi)
# Akurasi hmm lumayan

6223 2151 0.6543467780813113


Akurasi hmm lumayan. KIta baca dulu dari deskripsi

 ##### Percobaan Mengisi Jumlah Lantai dari kolom `Deskripsi`

In [92]:
df[df['Jumlah Lantai'].isna()==True]['Tipe Properti'].value_counts()

Rumah    1405
Name: Tipe Properti, dtype: int64

In [93]:
## Fungsi membaca jumlah lantai 
def JLantai(text):
    angka1='o'
    cari1 = re.search('lantai',text)
    try:
        S1 = cari1.span()
        hasil1 = text[S1[1]-12:S1[1]]
        H = re.findall(r"[^\n]+",hasil1)
        for i in H:
            if 'lantai' in i:
                hasil1 = i
                break
        x = re.search(r'\d\.\d|\d\,\d',hasil1)
        if x != None:
            angka1 = x.group()
        else:
            H = re.findall(r"[^.,]+",hasil1)
            for i in H:
                if 'lantai' in i:
                    hasil1 = i
                    break
            try:
                angka1 = (re.search(r'\d',hasil1)).group()
            except:
                S1 = cari1.span()
                hasil1 = text[S1[1]-6:S1[1]+7]
                H = re.findall(r"[^.,\n]+",hasil1)
                for i in H:
                    if 'lantai' in i:
                        hasil1 = i
                        break
                try:
                    cari = re.search("lantai",hasil1)
                    S = cari.span()
                    if ':' in hasil1[S[1]:S[1]+3]:
                        angka1 = (re.search(r'\d',hasil1)).group()
                except:
                    angka1 = 'ga ada angka?' 
    except:
        hasil1 = 'ga nemu'
    if angka1[0].isdigit() == False:    
        angka2='o'
        cari2 = re.search("tingkat",text)
        try:
            S2 = cari2.span()
            hasil2 = text[S2[1]-13:S2[1]]
            H = re.findall(r"[^\n]+",hasil2)
            for i in H:
                if 'tingkat' in i:
                    hasil2 = i
                    break
            x = re.search(r'\d\.\d|\d\,\d',hasil2)
            if x != None:
                angka2 = x.group()
            else:
                H = re.findall(r"[^.,]+",hasil2)
                for i in H:
                    if 'tingkat' in i:
                        hasil2 = i
                        break
                try:
                    angka2 = (re.search(r'\d',hasil2)).group()
                except:
                    S2 = cari2.span()
                    hasil2 = text[S2[1]-7:S2[1]+7]
                    H = re.findall(r"[^.,\n]+",hasil2)
                    for i in H:
                        if 'tingkat' in i:
                            hasil2 = i
                            break
                    try:
                        cari = re.search("tingkat",hasil2)
                        S = cari.span()
                        if ':' in hasil2[S[1]:S[1]+3]:
                            angka2 = (re.search(r'\d',hasil2)).group()
                    except:
                        angka2 = 'ga ada angka?' 
        except:
            hasil2 = 'ga nemu'
        if angka2[0].isdigit() == False:   
            angka3='o'
            cari3 = re.search("floors",text)
            try:
                S3 = cari3.span()
                hasil3 = text[S3[1]-12:S3[1]]
                H = re.findall(r"[^\n]+",hasil3)
                for i in H:
                    if 'floors' in i:
                        hasil3 = i
                        break
                x = re.search(r'\d\.\d|\d\,\d',hasil3)
                if x != None:
                    angka3 = x.group()
                else:
                    H = re.findall(r"[^.,]+",hasil3)
                    for i in H:
                        if 'floors' in i:
                            hasil3 = i
                            break
                    try:
                        angka3 = (re.search(r'\d',hasil3)).group()
                    except:
                        S3 = cari3.span()
                        hasil3 = text[S3[1]-6:S3[1]+7]
                        H = re.findall(r"[^.,\n]+",hasil3)
                        for i in H:
                            if 'floors' in i:
                                hasil3 = i
                                break
                        try:
                            cari = re.search("floors",hasil3)
                            S = cari.span()
                            if ':' in hasil3[S[1]:S[1]+3]:
                                angka3 = (re.search(r'\d',hasil3)).group()
                        except:
                            angka3 = 'ga ada angka?' 
            except:
                hasil3 = 'ga nemu'
            if angka3[0].isdigit() == False: 
                return np.nan
            else:
                return angka3
        else:
            return angka2
    else:
        return angka1

###########

In [95]:
# ddummy = df.copy()
# D = ddummy[ddummy['Jumlah Lantai'].isna()==True]

# for i in range(2):
#     print(JLantai(D.iloc[i]['Deskripsi'].lower()))
#     print(D.iloc[i]['Deskripsi'])
#     print(100*'=')

In [94]:
## Fungsi Convert JLantai
def konv(text):
    try:
        x = text.replace(',','.')
        return float(x)
    except:
        return text
# konv(np.nan)
# konv('1,5')

In [97]:
df['Jumlah Lantai_Desc'] = df['Jumlah Lantai']
df['Jumlah Lantai_Desc'] = df['Jumlah Lantai_Desc'].fillna(-1)
df['Jumlah Lantai_Desc'].value_counts()

2.0     3963
-1      1405
1       1302
1.0     1035
3.0     1013
4.0      191
5.0       16
7.0        2
40.0       1
25.0       1
8.0        1
Name: Jumlah Lantai_Desc, dtype: int64

In [98]:
df[df['Jumlah Lantai_Desc']==-1]

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Sertifikat,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc,garasi,Jumlah Lantai_copy,Jumlah Lantai_Desc
23,2.500000e+10,Menteng,566.0,685.0,18.0,6.0,NaN,Rumah Tua Terawat di Cikini Jakarta Pusat:Luas...,hos8214643,Rumah,...,HGB - Hak Guna Bangunan,NaN,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cikini, Jakarta Pusat",0,0.0,NaN,-1
56,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,NaN,DHYANA RUMAH KOS MEWAH LOKASI STRATEGIS DI SEN...,hos8213688,Rumah,...,"Lainnya (PPJB,Girik,Adat,dll)",NaN,29 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat",0,0.0,NaN,-1
59,7.500000e+09,Gambir,150.0,360.0,4.0,3.0,2.0,"petojo selatan, jakarta pusat,dibawah njop, lu...",hos8108638,Rumah,...,"Lainnya (PPJB,Girik,Adat,dll)",NaN,20 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Gambir, Jakarta Pusat",0,2.0,NaN,-1
85,1.250000e+10,Sawah Besar,600.0,474.0,4.0,4.0,NaN,RUMAH BAGUS DI GUNUNG SAHARI JAKARTA PUSAT:Rum...,hos8031743,Rumah,...,"Lainnya (PPJB,Girik,Adat,dll)",NaN,19 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Gunung Sahari, Jakarta Pusat",0,0.0,NaN,-1
109,5.500000e+10,Menteng,950.0,754.0,3.0,3.0,NaN,RUMAH MENTENG LUXURY SEMI FURNISH SIAP HUNI PR...,hos8223243,Rumah,...,SHM - Sertifikat Hak Milik,33000,7 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Menteng, Jakarta Pusat",0,0.0,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,6.500000e+09,Mampang Prapatan,300.0,200.0,4.0,4.0,NaN,"Rumah Brand New di Kemang, Jakarta Selatan ~ S...",hos6806910,Rumah,...,SHM - Sertifikat Hak Milik,NaN,30 Juni 2020,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Kemang, Jakarta Selatan",0,0.0,NaN,-1
9738,2.200000e+10,Mampang Prapatan,650.0,1050.0,4.0,4.0,2.0,"Rumah di Kemang, Jakarta Selatan ~ Pool ~ Tama...",hos6807176,Rumah,...,SHM - Sertifikat Hak Milik,NaN,30 Juni 2020,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Kemang, Jakarta Selatan",0,2.0,NaN,-1
9856,1.550000e+10,Mampang Prapatan,450.0,507.0,5.0,5.0,2.0,"Rumah Mewah di Kemang, Jakarta Selatan ~ Taman...",hos7600570,Rumah,...,SHM - Sertifikat Hak Milik,NaN,25 November 2020,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Kemang, Jakarta Selatan",0,2.0,NaN,-1
9857,3.200000e+09,Tebet,150.0,135.0,3.0,2.0,NaN,"Rumah Minimalis di Tebet, Jakarta Selatan ~ Si...",hos7609887,Rumah,...,SHM - Sertifikat Hak Milik,NaN,28 November 2020,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Tebet, Jakarta Selatan",0,0.0,NaN,-1


In [99]:
for i in range(len(df)):
    if df['Jumlah Lantai_Desc'].iloc[i]==-1:
        text = df['Deskripsi'].iloc[i]
        y = konv(JLantai(text))
        df['Jumlah Lantai_Desc'].iloc[i] = y
df['Jumlah Lantai_Desc'].value_counts()

2.0     4121
1       1302
1.0     1067
3.0     1065
4.0      207
2.5       25
5.0       22
3.5       14
1.5       10
7.0        2
1.3        2
8.0        2
0.0        2
25.0       1
4.5        1
40.0       1
2.2        1
Name: Jumlah Lantai_Desc, dtype: int64

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   kt                    8930 non-null   float64
 5   km                    8930 non-null   float64
 6   car                   3950 non-null   float64
 7   Deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   Tipe Properti         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  Jumlah Lantai         7525 non-null   object 
 13  Kamar Pembantu        4080 non-null   float64
 14  Kamar Mandi Pembantu  3703 non-null   float64
 15  Sertifikat           

In [102]:
df['Jumlah Lantai_Desc'] = df['Jumlah Lantai_Desc'].astype(float)
df['Jumlah Lantai_Desc'].value_counts()

2.0     4121
1.0     2369
3.0     1065
4.0      207
2.5       25
5.0       22
3.5       14
1.5       10
1.3        2
0.0        2
7.0        2
8.0        2
4.5        1
25.0       1
2.2        1
40.0       1
Name: Jumlah Lantai_Desc, dtype: int64

In [109]:
df['Jumlah Lantai_Desc'] = df['Jumlah Lantai_Desc'].fillna(-1)
df['Jumlah Lantai_Desc'] = df['Jumlah Lantai_Desc'].apply(lambda x : int(x))
df['Jumlah Lantai_Desc'].value_counts()

 2     4147
 1     2381
-1     1085
 3     1079
 4      208
 5       22
 7        2
 8        2
 0        2
 25       1
 40       1
Name: Jumlah Lantai_Desc, dtype: int64

In [110]:
# ## Percobaan Mengisi Jumlah Lantai Menggunakan Ceil
# ddummy = A
# Ceil = ddummy[ddummy['Tipe Properti'] == 'Rumah']
# Ceil = Ceil.dropna(subset=['Jumlah Lantai', 'lb', 'lt'])
# Ceil = Ceil[['lb','lt','Jumlah Lantai']]
# # Ceil = Ceil.astype(int)
# Ceil['ceil'] = Ceil['lb']/Ceil['lt']
# Ceil['ceil'] = Ceil['ceil'].apply(lambda x : math.ceil(x))
# x = len(Ceil)
# y = len(Ceil[Ceil['Jumlah Lantai'] != Ceil['ceil']])
# akurasi = (x-y)/x
# print(x,y,akurasi)
# # Akurasi hmm lumayan

### Munging Kolom `listrik`

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   kt                    8930 non-null   float64
 5   km                    8930 non-null   float64
 6   car                   3950 non-null   float64
 7   Deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   Tipe Properti         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  Jumlah Lantai         7525 non-null   object 
 13  Kamar Pembantu        4080 non-null   float64
 14  Kamar Mandi Pembantu  3703 non-null   float64
 15  Sertifikat           

In [113]:
def listrik(df):
    df_copy = df.copy()
    df_copy[''] = np.zeros(len(df_copy))
    df_copy['Deskripsi'] = df_copy['Deskripsi'].fillna('aaaa')
    for i in range(len(df_copy)):
        text = str(df_copy['Deskripsi'].iloc[i]).lower()
        try:
            cari = re.search('listrik',text)
            S = cari.span()
            txt = text[S[1]:S[1]+10]
            clean = ''.join(re.findall(r'[^\D]+',txt))
            df_copy['Daya Listrik'].iloc[i] = clean
        except:
            pass
    return df_copy
listrik(df)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc,garasi,Jumlah Lantai_copy,Jumlah Lantai_Desc,
0,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,1300,6 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat",0,100.0,3.0,3,0.0
1,1.750000e+09,Cempaka Putih,97.0,49.0,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,2200,27 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Putih, Jakarta Pusat",0,0.0,3.0,3,0.0
2,1.500000e+09,Kemayoran,110.0,70.0,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,...,2200,10 Mei 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Sumur Batu, Jakarta Pusat",0,1.0,2.0,2,0.0
3,1.750000e+09,Senen,150.0,84.0,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,...,NaN,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Senen, Jakarta Pusat",0,0.0,2.0,2,0.0
4,1.375000e+09,Cempaka Putih,65.0,35.0,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,...,2200,26 April 2021,https://www.rumah123.com/properti/jakarta-pusa...,NaN,"Cempaka Putih, Jakarta Pusat",0,0.0,2.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9863,1.100000e+09,Jagakarsa,100.0,63.0,3.0,2.0,1.0,Hanya 5 Unit siapa cepat dia dapat:''Hanya 5 u...,hos7846972,Rumah,...,2200,29 Januari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan",0,1.0,2.0,2,0.0
9864,1.350000e+09,Jagakarsa,110.0,95.0,3.0,3.0,1.0,Rumah Cluster Indent siap Bangun:Hunian Cluste...,hos7878697,Rumah,...,2200,5 Februari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan",0,1.0,2.0,2,0.0
9865,1.550000e+09,Jagakarsa,130.0,95.0,3.0,3.0,2.0,Cluster exclusive Best area kebagusan:*Hunian ...,hos7875576,Rumah,...,2200,4 Februari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan",0,2.0,2.0,2,0.0
9866,1.350000e+09,Jagakarsa,130.0,86.0,3.0,3.0,1.0,Rumah Cluster Jakarta selatan jagakarsa:Hunian...,hos7861060,Rumah,...,1300,2 Februari 2021,https://www.rumah123.com/properti/jakarta-sela...,NaN,"Jagakarsa, Jakarta Selatan",0,1.0,2.0,2,0.0


In [114]:
df = listrik(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   kt                    8930 non-null   float64
 5   km                    8930 non-null   float64
 6   car                   3950 non-null   float64
 7   Deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   Tipe Properti         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  Jumlah Lantai         7525 non-null   object 
 13  Kamar Pembantu        4080 non-null   float64
 14  Kamar Mandi Pembantu  3703 non-null   float64
 15  Sertifikat           

In [115]:
df.sample(1)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,unkown_loc,garasi,Jumlah Lantai_copy,Jumlah Lantai_Desc,
2121,1.039000e+09,Ciracas,70.0,77.0,3.0,2.0,1.0,Rumah dengan 2 akses tol:Pesona Cibubur Reside...,hos8108911,Rumah,...,1300,28 April 2021,https://www.rumah123.com/properti/jakarta-timu...,NaN,"Cibubur, Jakarta Timur",0,1.0,2.0,2,0.0


In [120]:
df = df.drop(columns=['unkown_loc', ''])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   kt                    8930 non-null   float64
 5   km                    8930 non-null   float64
 6   car                   3950 non-null   float64
 7   Deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   Tipe Properti         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  Jumlah Lantai         7525 non-null   object 
 13  Kamar Pembantu        4080 non-null   float64
 14  Kamar Mandi Pembantu  3703 non-null   float64
 15  Sertifikat           

### Munging Kolom `Sertifikat`

In [121]:
df['Sertifikat'].unique()

array(['SHM - Sertifikat Hak Milik', 'Lainnya (PPJB,Girik,Adat,dll)',
       'HGB - Hak Guna Bangunan', 'HS - Hak Sewa', nan, 'HP - Hak Pakai'],
      dtype=object)

In [122]:
df['ada_sertifikat'] = df['Sertifikat']
df['ada_sertifikat'] = df['ada_sertifikat'].fillna(0)
df.loc[df['ada_sertifikat'] != 0, "ada_sertifikat"] = 1
df['ada_sertifikat'].value_counts()

1    7955
0     975
Name: ada_sertifikat, dtype: int64

In [123]:
def adasertifikat(text):
    text = text.lower()
    if 'sertifikat' in text or 'surat' in text or 'hgb' in text or 'shm' in text or 'ppjb' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    if df['ada_sertifikat'].iloc[i] == 0:
        df['ada_sertifikat'].iloc[i] = adasertifikat(df['Deskripsi'].iloc[i])    

df['ada_sertifikat'].value_counts()

1    8415
0     515
Name: ada_sertifikat, dtype: int64

### Munging Kolom `Fasilitas`

In [125]:
df['Fasilitas'].value_counts()

AC, Carport, Swimming Pool, Garden                                                                                                                                                                                         70
AC                                                                                                                                                                                                                         68
Air Conditioning                                                                                                                                                                                                           56
AC, Swimming Pool, Garden                                                                                                                                                                                                  45
AC, Swimming Pool                                                                                               

In [128]:
df['Fasilitas'] = df['Fasilitas'].fillna('tidak_ada')

In [129]:
### Kolom gym
df['gym'] = 0
df.sample()

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Daya Listrik,Tanggal Tayang,url,Fasilitas,lokasi_copy,garasi,Jumlah Lantai_copy,Jumlah Lantai_Desc,ada_sertifikat,gym
1460,1.100000e+10,Tanah Abang,150.0,219.0,11.0,7.0,2.0,Rumah Hitung Tanah di Pejompongan:Lokasi strat...,hos6993795,Rumah,...,NaN,22 Juli 2020,https://www.rumah123.com/properti/jakarta-pusa...,tidak_ada,"Bendungan Hilir, Jakarta Pusat",2.0,NaN,-1,1,0


In [130]:
def adagym(text):
    text = text.lower()
    if 'gym' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    df['gym'].iloc[i] = adagym(df['Deskripsi'].iloc[i])
    if df['gym'].iloc[i] == 0:
        df['gym'].iloc[i] = adagym(df['Fasilitas'].iloc[i])    

df['gym'].value_counts()

0    8663
1     267
Name: gym, dtype: int64

In [131]:
## Kolom taman
df['taman'] = 0
def adataman(text):
    text = text.lower()
    if 'taman' in text or 'garden' in text or 'halaman' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    df['taman'].iloc[i] = adataman(df['Deskripsi'].iloc[i])
    if df['taman'].iloc[i] == 0:
        df['taman'].iloc[i] = adataman(df['Fasilitas'].iloc[i])    

df['taman'].value_counts()

0    6618
1    2312
Name: taman, dtype: int64

In [132]:
## Kolom pool
df['pool'] = 0
def adapool(text):
    text = text.lower()
    if 'pool' in text or 'kolam renang' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    df['pool'].iloc[i] = adapool(df['Deskripsi'].iloc[i])
    if df['pool'].iloc[i] == 0:
        df['pool'].iloc[i] = adapool(df['Fasilitas'].iloc[i])    

df['pool'].value_counts()

0    7232
1    1698
Name: pool, dtype: int64

In [133]:
df.sample(1)

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,url,Fasilitas,lokasi_copy,garasi,Jumlah Lantai_copy,Jumlah Lantai_Desc,ada_sertifikat,gym,taman,pool
5183,1.300000e+09,Cengkareng,100.0,78.0,3.0,2.0,1.0,cepat rumah 2 lt di Kosambi Baru Duri Kosambi ...,hos8231083,Rumah,...,https://www.rumah123.com/properti/jakarta-bara...,tidak_ada,"Duri Kosambi, Jakarta Barat",1.0,2.0,2,1,0,0,0


In [134]:
## Kolom SHM
df['shm'] = 0
def adashm(text):
    text = text.lower()
    if 'shm' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    df['shm'].iloc[i] = adashm(df['Deskripsi'].iloc[i])
    if df['shm'].iloc[i] == 0:
        df['shm'].iloc[i] = adashm(df['Fasilitas'].iloc[i])    

df['shm'].value_counts()

0    4899
1    4031
Name: shm, dtype: int64

In [135]:
## Kolom hgb
df['hgb'] = 0
def adahgb(text):
    text = text.lower()
    if 'hgb' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    df['hgb'].iloc[i] = adahgb(df['Deskripsi'].iloc[i])
    if df['hgb'].iloc[i] == 0:
        df['hgb'].iloc[i] = adahgb(df['Fasilitas'].iloc[i])    

df['hgb'].value_counts()

0    8611
1     319
Name: hgb, dtype: int64

In [136]:
## Kolom hgb
df['imb'] = 0
def adaimb(text):
    text = text.lower()
    if 'imb' in text:
        return 1
    else:
        return 0
    
for i in range(len(df)):
    df['imb'].iloc[i] = adaimb(df['Deskripsi'].iloc[i])
    if df['imb'].iloc[i] == 0:
        df['imb'].iloc[i] = adaimb(df['Fasilitas'].iloc[i])    

df['imb'].value_counts()

0    7502
1    1428
Name: imb, dtype: int64

### Membuat Kolom `Source`

In [137]:
df['source'] = 'rumah123'

In [138]:
df.head()

,harga,lokasi,lb,lt,kt,km,car,Deskripsi,ID Iklan,Tipe Properti,...,Jumlah Lantai_copy,Jumlah Lantai_Desc,ada_sertifikat,gym,taman,pool,shm,hgb,imb,source
0,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,3.0,3,1,0,0,0,1,0,1,rumah123
1,1.750000e+09,Cempaka Putih,97.0,49.0,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,3.0,3,1,0,1,0,1,0,0,rumah123
2,1.500000e+09,Kemayoran,110.0,70.0,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,...,2.0,2,1,0,0,0,1,0,1,rumah123
3,1.750000e+09,Senen,150.0,84.0,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,...,2.0,2,1,0,0,0,0,0,1,rumah123
4,1.375000e+09,Cempaka Putih,65.0,35.0,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,...,2.0,2,1,0,0,0,1,0,0,rumah123


### Mengubah nama kolom

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   harga                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   kt                    8930 non-null   float64
 5   km                    8930 non-null   float64
 6   car                   3950 non-null   float64
 7   Deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   Tipe Properti         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  Jumlah Lantai         7525 non-null   object 
 13  Kamar Pembantu        4080 non-null   float64
 14  Kamar Mandi Pembantu  3703 non-null   float64
 15  Sertifikat           

In [145]:
df['price_conditions'] = np.nan
df['year_built'] = np.nan
df['empty'] = np.nan
df['rooms_total'] = np.nan
df['deposit'] = np.nan
df['terdekat'] = np.nan
df['building_size'] = df['lb']
df['land_size'] = df['lt']
df['kamar'] = df['kt']
df['kamar_mandi'] = df['km']

In [148]:
df = df.rename(columns={'km': 'bathrooms', 'kt': 'bedrooms', 'Deskripsi' : 'deskripsi',
                    'Fasilitas' : 'fasilitas', 'harga' : 'price', 
                    'car' : 'car_spaces',
                    'Jumlah Lantai_Desc' : 'floors_total', 'Kamar Mandi Pembantu' : 'maidbathrooms',
                    'Kamar Pembantu' : 'maidrooms', 'Jumlah Lantai' : 'floors', 'Daya Listrik' : 'electricity',
                   'garasi' : 'ada_garasi', 'Tipe Properti' : 'tipe_property', 'Sertifikat' : 'jenis_sertifikat',
                   'ada_sertifikat' : 'sertifikat'
                  })

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   bedrooms              8930 non-null   float64
 5   bathrooms             8930 non-null   float64
 6   car_spaces            3950 non-null   float64
 7   deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   tipe_property         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  floors                7525 non-null   object 
 13  maidrooms             4080 non-null   float64
 14  maidbathrooms         3703 non-null   float64
 15  jenis_sertifikat     

In [150]:
df.drop(columns=['Jumlah Lantai_copy'],inplace=True)

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8930 entries, 0 to 9867
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   bedrooms              8930 non-null   float64
 5   bathrooms             8930 non-null   float64
 6   car_spaces            3950 non-null   float64
 7   deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   tipe_property         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  floors                7525 non-null   object 
 13  maidrooms             4080 non-null   float64
 14  maidbathrooms         3703 non-null   float64
 15  jenis_sertifikat     

In [159]:
df['sertifikat'] = df['sertifikat'].astype(int)
# df['electricity'] = df['electricity'].astype(float)

In [169]:
# print(df[df['electricity']=='Lainnya'].iloc[2]['deskripsi'])

In [176]:
df.loc[df['electricity'] == "Lainnya", "electricity"] = -1
# df['electricity'] = df['electricity'].astype(float)

['1300' '2200' 0 '16500' '13200' '10600' '3300' '16' '5500' '3500' '13300'
 '' '23000' '17000' '33000' '10000' '4400' '11000' '38100' '4000' '13500'
 '7700' '66000' '6600' '26000' '12700' -1 '2' '10800' '51' '22000' '12000'
 '7500' '900' '55003' '25000' '24000' '53000' '10' '7600' '440022' '2500'
 '19000' '18000' '3200' '1056' '3' '16400' '13002' '660' '16000' '30800'
 '50000' '23500' '12' '8000' '3520' '10613' '800' '22005' '3000' '1200'
 '40000' '44' '22' '1500' '1' '14500' '17600' '9000' '106' '9900' '330001'
 '450' '4500' '21000' '350022' '5' '5000' '30500' '95000' '27500' '13900'
 '440035' '10500' '2300' '13000' '130' '9500' '44003500' '2220' '2400'
 '7000' '20000' '244' '220' '12500' '3330' '15000' '8090' '90' '22002'
 '110005' '350' '55' '15400' '42' '44004' '11' '41500' '18500' '30000'
 '88004' '77007' '85000' '44005' '21200' '30' '550' '6500' '11500' '14000'
 '770' '35' '22001' '61000' '76000' '220012' '10700' '16100' '55004'
 '120001' '55005' '35000' '12300' '3600' '6' '15500

In [190]:
x = df['electricity']
df.head(10)
# for i in range(len(x)):
#     print(i, x[i], type(x[i]))

,price,lokasi,lb,lt,bedrooms,bathrooms,car_spaces,deskripsi,ID Iklan,tipe_property,...,price_conditions,year_built,empty,rooms_total,deposit,terdekat,building_size,land_size,kamar,kamar_mandi
0,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,1553.0,774.0,106.0,51.0
1,1.750000e+09,Cempaka Putih,97.0,49.0,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,97.0,49.0,2.0,1.0
2,1.500000e+09,Kemayoran,110.0,70.0,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,110.0,70.0,3.0,2.0
3,1.750000e+09,Senen,150.0,84.0,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,150.0,84.0,12.0,12.0
4,1.375000e+09,Cempaka Putih,65.0,35.0,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,65.0,35.0,2.0,1.0
5,7.700000e+10,Menteng,2000.0,760.0,7.0,7.0,NaN,Rumah Mewah nan Luas di Menteng central:Dijual...,hos6051656,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,760.0,7.0,7.0
8,8.650000e+08,Sawah Besar,54.0,29.0,2.0,1.0,NaN,Rumah Minimalis di Gunung Sahari Jakarta Pusat...,hos8273483,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,54.0,29.0,2.0,1.0
9,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,NaN,CHANDRA*rumah kost bagus lt 774m2 lokasi strat...,hos8212207,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,1553.0,774.0,106.0,51.0
10,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,NaN,"Beli Kost Bonus 1 Kost Lagi, Mewah, Occupancy ...",hos8264752,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,1553.0,774.0,106.0,51.0
11,1.445000e+09,Sawah Besar,89.0,44.0,3.0,2.0,NaN,rumah minimalis gunung sahari dekat dengan pus...,hos8236689,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,89.0,44.0,3.0,2.0


In [195]:
df = df.reset_index(drop=True)
df.head(10)

,price,lokasi,lb,lt,bedrooms,bathrooms,car_spaces,deskripsi,ID Iklan,tipe_property,...,price_conditions,year_built,empty,rooms_total,deposit,terdekat,building_size,land_size,kamar,kamar_mandi
0,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,1553.0,774.0,106.0,51.0
1,1.750000e+09,Cempaka Putih,97.0,49.0,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,97.0,49.0,2.0,1.0
2,1.500000e+09,Kemayoran,110.0,70.0,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,110.0,70.0,3.0,2.0
3,1.750000e+09,Senen,150.0,84.0,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,150.0,84.0,12.0,12.0
4,1.375000e+09,Cempaka Putih,65.0,35.0,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,65.0,35.0,2.0,1.0
5,7.700000e+10,Menteng,2000.0,760.0,7.0,7.0,NaN,Rumah Mewah nan Luas di Menteng central:Dijual...,hos6051656,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,760.0,7.0,7.0
6,8.650000e+08,Sawah Besar,54.0,29.0,2.0,1.0,NaN,Rumah Minimalis di Gunung Sahari Jakarta Pusat...,hos8273483,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,54.0,29.0,2.0,1.0
7,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,NaN,CHANDRA*rumah kost bagus lt 774m2 lokasi strat...,hos8212207,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,1553.0,774.0,106.0,51.0
8,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,NaN,"Beli Kost Bonus 1 Kost Lagi, Mewah, Occupancy ...",hos8264752,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,1553.0,774.0,106.0,51.0
9,1.445000e+09,Sawah Besar,89.0,44.0,3.0,2.0,NaN,rumah minimalis gunung sahari dekat dengan pus...,hos8236689,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,89.0,44.0,3.0,2.0


In [197]:
x = df['electricity']
# df.head(10)
for i in range(len(x)):
    print(i, x[i], type(x[i]))

0 1300 <class 'str'>
1 2200 <class 'str'>
2 2200 <class 'str'>
3 0 <class 'int'>
4 2200 <class 'str'>
5 0 <class 'int'>
6 2200 <class 'str'>
7 0 <class 'int'>
8 16500 <class 'str'>
9 2200 <class 'str'>
10 0 <class 'int'>
11 2200 <class 'str'>
12 2200 <class 'str'>
13 13200 <class 'str'>
14 2200 <class 'str'>
15 2200 <class 'str'>
16 0 <class 'int'>
17 0 <class 'int'>
18 10600 <class 'str'>
19 0 <class 'int'>
20 0 <class 'int'>
21 0 <class 'int'>
22 2200 <class 'str'>
23 2200 <class 'str'>
24 2200 <class 'str'>
25 2200 <class 'str'>
26 3300 <class 'str'>
27 16 <class 'str'>
28 0 <class 'int'>
29 0 <class 'int'>
30 5500 <class 'str'>
31 3500 <class 'str'>
32 0 <class 'int'>
33 2200 <class 'str'>
34 2200 <class 'str'>
35 2200 <class 'str'>
36 2200 <class 'str'>
37 13300 <class 'str'>
38  <class 'str'>
39 10600 <class 'str'>
40 2200 <class 'str'>
41 2200 <class 'str'>
42 2200 <class 'str'>
43 2200 <class 'str'>
44 0 <class 'int'>
45 23000 <class 'str'>
46 0 <class 'int'>
47 3500 <class 'st

In [198]:
df.loc[df['electricity'] == "", "electricity"] = -1

In [199]:
df['electricity'] = df['electricity'].astype(float)

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8930 entries, 0 to 8929
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 8930 non-null   float64
 1   lokasi                8930 non-null   object 
 2   lb                    8930 non-null   float64
 3   lt                    7628 non-null   float64
 4   bedrooms              8930 non-null   float64
 5   bathrooms             8930 non-null   float64
 6   car_spaces            3950 non-null   float64
 7   deskripsi             8930 non-null   object 
 8   ID Iklan              8930 non-null   object 
 9   tipe_property         8930 non-null   object 
 10  Dilengkapi Perabotan  5565 non-null   object 
 11  Kondisi Properti      6350 non-null   object 
 12  floors                7525 non-null   object 
 13  maidrooms             4080 non-null   float64
 14  maidbathrooms         3703 non-null   float64
 15  jenis_sertifikat     

### df to csv


In [201]:
path = r'C:\Users\dosal\Downloads\Buat_Dede\My_Udemy_BuatDede\Web_Scrapping_with_Python_With_BeautifulSoup_and_Selenium_2021\file_scrapping_proyek_harga_rumah\Stage_1_Data_Munging\munged_data_rumah123.csv'
df.to_csv(path, index = False, header=True)

In [202]:
pd.read_csv('munged_data_rumah123.csv')

,price,lokasi,lb,lt,bedrooms,bathrooms,car_spaces,deskripsi,ID Iklan,tipe_property,...,price_conditions,year_built,empty,rooms_total,deposit,terdekat,building_size,land_size,kamar,kamar_mandi
0,2.500000e+10,Senen,1553.0,774.0,106.0,51.0,100.0,"Guys, ada Kos2an mewah plus ada bonus keren di...",hos8220289,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,1553.0,774.0,106.0,51.0
1,1.750000e+09,Cempaka Putih,97.0,49.0,2.0,1.0,NaN,rumah minimalis cempaka putih tengah strategis...,hos8215191,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,97.0,49.0,2.0,1.0
2,1.500000e+09,Kemayoran,110.0,70.0,3.0,2.0,1.0,Rumah bagus minimalis siap huni di Sumur Batu ...,hos8253785,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,110.0,70.0,3.0,2.0
3,1.750000e+09,Senen,150.0,84.0,12.0,12.0,NaN,"KOST 12KT 12KM, di Belakang Stasiun Senen Jaka...",hos8071827,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,150.0,84.0,12.0,12.0
4,1.375000e+09,Cempaka Putih,65.0,35.0,2.0,1.0,NaN,Rumah Murah Minimalis Mewah &amp; Elegant di C...,hos8225178,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,65.0,35.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8925,1.100000e+09,Jagakarsa,100.0,63.0,3.0,2.0,1.0,Hanya 5 Unit siapa cepat dia dapat:''Hanya 5 u...,hos7846972,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,63.0,3.0,2.0
8926,1.350000e+09,Jagakarsa,110.0,95.0,3.0,3.0,1.0,Rumah Cluster Indent siap Bangun:Hunian Cluste...,hos7878697,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,110.0,95.0,3.0,3.0
8927,1.550000e+09,Jagakarsa,130.0,95.0,3.0,3.0,2.0,Cluster exclusive Best area kebagusan:*Hunian ...,hos7875576,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,130.0,95.0,3.0,3.0
8928,1.350000e+09,Jagakarsa,130.0,86.0,3.0,3.0,1.0,Rumah Cluster Jakarta selatan jagakarsa:Hunian...,hos7861060,Rumah,...,NaN,NaN,NaN,NaN,NaN,NaN,130.0,86.0,3.0,3.0
